In [1]:
# pandas data analysis library
import pandas as pd

# regular expressions library
import re

In [2]:
# read csv file containing webscraper output
df = pd.read_csv('../data/webscrapper_output.csv')

Split Dataframe into variance and no variance

In [3]:
# determines whether a product is a duplicate based on the product's name
variant_products = (df['productname'].duplicated(keep=False))

# '~' makes it so only false values are returned
no_variant_products = ~variant_products

In [4]:
# assign df to products with no variance
df = df[no_variant_products]

Add Domain To Image Links

In [5]:
# this function adds website domain to image links
def add_domain_to_image_links(image_link):

    domain = 'www.wonatrading.com/'

    # if image link is a string add domain
    if isinstance(image_link, str):
        return domain + image_link

    # otherwise return 
    return ''

In [6]:
# apply add_domain_to_image_links to both image columns 
df['image1'] = df['image1'].apply(add_domain_to_image_links)
df['image2'] = df['image2'].apply(add_domain_to_image_links)

Clean Price Column

In [7]:
# this function returns the first price found in price column
def clean_price(price):

    pattern = re.compile(r'\d.{2}\d')  # pattern we're looking for

    matches = pattern.finditer(price)  # number of matches found

    matches_list = [match.group() for match in matches]

    return matches_list[0]

In [8]:
df['price'] = df['price'].apply(clean_price) 

Combine all 3 category columns into 1 Column

In [9]:
# combine each category column into 1
# JEWELRY;ANKLET/FASHION
df['category'] = df['category1'] + ';' + df['category2'] + '/' + df['category3']

# delete old category columns
df.drop(['category1', 'category2', 'category3'], axis=1, inplace=True)

Clean Description Functions

In [10]:
def remove_name_from_product_description(description):

    # r'\s' + adds a white space to the begining of the pattern
    pattern = r'\s' + df['productname']  # pattern we're looking for

    replacement = ''  # what to replace the string with

    target = description  # the string we want to replace

    # re.sub replaces the string and ignorescase
    replaced = re.sub(pattern, replacement, target, flags=re.IGNORECASE)

    # replace description of each product in original file
    return replaced

def remove_style_number_from_product_description(description):

    pattern = r'( Style No : )\d+\s' # pattern we're looking for

    replacement = ''  # replace pattern with nothing

    target = description  # the string we want to replace

    # re.sub replaces the string and ignorescase
    replaced = re.sub(pattern, replacement, target, flags=re.IGNORECASE)

    # return the cleaned string
    return replaced

def remove_colors_from_descrition(description):
    
    # pattern we're looking for
    # 2 patterns
    # (Color : )\w+\s OR (Color : )\w+,\s*\w+\s*
    pattern = r'((Color : )\w+\s|(Color : )\w+,\s*\w+\s*)'    

    replacement = ''             # replace pattern with nothing
    target = description         # the string we want to replace

    # re.sub replaces the string and ignorescase
    replaced = re.sub(pattern, replacement, target, flags=re.IGNORECASE)

    return replaced # return the cleaned string

(Clean Description) Dataframe Description Column = result of all the functions

In [11]:
# remove style number from description
df['description'] = df['description'].apply(remove_style_number_from_product_description)

# remove colors from description
df['description'] = df['description'].apply(remove_colors_from_descrition)

# list of all product names
productname_list = df['productname'].tolist()

# remove product names from product description column
df['description'] = df['description'].str.replace('|'.join(productname_list), '')

# prints entire column width
pd.set_option('display.max_colwidth', len(df['description']))

# print description column
df['description']

0    Theme : Pearl  Size : 0.25"H, 9" + 3" L  One Side Only Lead and Nickel Compliant            
1    Size : 0.1" H, 8.25" H, 2.5" L  One Side Only Lead and Nickel Compliant                     
8    Theme : Message  Size : 2.25" X 0.7", 8.25" + 3" L  One Side Only Lead and Nickel Compliant 
Name: description, dtype: object

Change column names to the column names in no variant template

In [12]:
# 2 lists containing the field names for input and outputs
input_columns = ['productname','category','image1','image2','description','price']
output_columns = ['Product Name', 'Category','Product Image File - 1','Product Image File - 2','Product Description','Price']

# keys are original column names, values are what they should be changed to
change_column_names_dictionary = dict(zip(input_columns, output_columns))

# this is the command changes column names in dataframe to column names that are in no variant template
# inplace=True means that it applies the change to original data frame
df.rename(columns=change_column_names_dictionary, inplace=True)

df

,Product Name,Product Image File - 1,Product Image File - 2,Product Description,Price,Category
0,Freshwater Pearl Disc Beaded Anklet,www.wonatrading.com/images/20200312/AK0001-@GD-NMLT1@025H-9_3L@479682@375@01.jpg,,"Theme : Pearl Size : 0.25""H, 9"" + 3"" L One Side Only Lead and Nickel Compliant",3.75,JEWELRY;ANKLET/FASHION
1,Bead Link Layered Anklet,www.wonatrading.com/images/20200123/AK0002-@GD-HEM@01H-825_25L@473115@375@01.jpg,,"Size : 0.1"" H, 8.25"" H, 2.5"" L One Side Only Lead and Nickel Compliant",3.75,JEWELRY;ANKLET/FASHION
8,Rhinestone Pave Boss Anklet,www.wonatrading.com/images/20191206/AK1503-@GD-CRY-BOSS@225X07-825_3L@467403@200@01.jpg,www.wonatrading.com/images/20191206/des_img/AK1503-@GD-CRY-BOSS@225X07-825_3L@467403@200@01@1.jpg,"Theme : Message Size : 2.25"" X 0.7"", 8.25"" + 3"" L One Side Only Lead and Nickel Compliant",2.00,JEWELRY;ANKLET/FASHION


In [13]:
# write dataframe to csv file
df.to_csv('output.csv', index=False)